## Solution 1:

In [ ]:
def withdraw(balance, amount) :
    try:
        if amount > balance:
            raise ValueError("Insufficient Funds.")
        return balance - amount

    except ValueError as err:
        print(err)
        return balance

## Solution 2:

In [ ]:
def calculate (num1, num2, operator) :
    if operator == '+':
        return num1 + num2
    elif operator == '-':
        return num1 - num2
    elif operator == '*':
        return num1 * num2
    elif operator == '/' and num2 != 0 :
        return num1 / num2
    elif operator == '%' :
        return num1 % num2

    else:
        return "Invalid Operator."

## Solution 3:

In [ ]:
students = ["Alice", "Bob", "Charlie", "David", "Eve"]
Students_A_D_Name = []

for SN in students:
    if SN.startswith('A') or SN.startswith('D'):
        Students_A_D_Name.append(SN)


In [ ]:
print(Students_A_D_Name)

## Solution 4:

In [ ]:
class Person:
    def introduce(self):
        print("I am a person.")

class Student(Person):
    def introduce(self):
        print("I am a student.")


In [ ]:
person = Person()
student = Student()

person.introduce()
student.introduce()

## Solution 7:

**Load the dataset**

In [ ]:
!pip install kagglehub pandas numpy nltk scikit-learn imblearn transformers tensorflow


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

#path
df = pd.read_csv(f"{path}/training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None)

# Rename
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
df = df[['sentiment', 'text']]

# Convert label
df['sentiment'] = df['sentiment'].replace({0: 0, 4: 1})

In [ ]:
import tensorflow as tf

# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

with tf.device('/GPU:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], dtype=tf.float32)
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], dtype=tf.float32)
    c = tf.matmul(a, b)




**Preprocessing**

In [ ]:

import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'[^a-zA-Z ]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

#after processing
df.head()


In [ ]:
from sklearn.model_selection import train_test_split

# take 80/20 ratio
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

print("Train Size:", len(X_train))
print("Test Size:", len(X_test))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF Train Shape:", X_train_tfidf.shape)
print("TF-IDF Test Shape:", X_test_tfidf.shape)


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_tfidf_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

#After appling smote
pd.Series(y_train_smote).value_counts()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# traditional ML model
models = {
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(kernel='linear', probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

# used matrics
for name, model in models.items():
    model.fit(X_train_tfidf_smote, y_train_smote)
    y_pred = model.predict(X_test_tfidf)

    print(f"\nModel: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_test, model.predict_proba(X_test_tfidf)[:,1]):.4f}")


In [ ]:
from sklearn.model_selection import GridSearchCV

#hyperparameter
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

# Initialize model and GridSearchCV
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train_tfidf_smote, y_train_smote)

# Best parameters
print("Best Parameters for Random Forest:", grid_search.best_params_)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Tokenization
max_words = 5000
max_len = 50
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

#lstm
model = Sequential([
    Embedding(max_words, 100, input_length=max_len),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train_seq, y_train, batch_size=32, epochs=5, validation_data=(X_test_seq, y_test))


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')
output = bert_model([input_ids, mask])[0]

model_bert = tf.keras.Model(inputs=[input_ids, mask], outputs=output)
model_bert.compile(optimizer=Adam(learning_rate=2e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(" DOne ")
